`Work was done 23.12.2019`

## О чём
### Ансамбли
В этом ноутбуке реализованы алгоритмы бустинга и бэггинга.

In [1]:
import numpy as np
import numpy.testing as npt
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.datasets import make_classification
from scipy.optimize import minimize
from scipy.special import expit

## 1. Сэмплирование случайных объектов и признаков

Во многих ансамблевых алгоритмах используется прием, заключающийся в обучении на случайной подвыборке объектов или на случайном подмножестве признаков.

Так что для начала реализуем класс, который будет упрощать семплирование различных подмассивов данных

In [2]:
class BaseSampler:
    def __init__(self, max_samples=1.0, bootstrap=False):
        """
        Parameters
        ----------
        bootstrap : Boolean
            if True then use bootstrap sampling
        max_samples : float in [0;1]
            proportion of sampled examples
        """
        self.bootstrap = bootstrap
        self.max_samples = max_samples
    
    def sample(self, x):
        raise NotImplementedError

class ObjectSampler(BaseSampler):
    def __init__(self, axis=0, max_samples=1.0, bootstrap=True):
        """
        Parameters
        ----------
        axis : int
            which axis use to sample
        """
        self.axis = axis
        super().__init__(max_samples=max_samples, bootstrap=bootstrap)
    
    def sample(self, x, y):
        """
        Parameters
        ----------
        x : numpy ndarray of shape (n_objects, n_features)
        y : numpy ndarray of shape (n_objects,)
        
        Returns
        -------
        x_sampled, y_sampled : numpy ndarrays of shape (n_samples, n_features) and (n_samples,)
            n_samples = x_sampled.shape[0] * self.max_samples
        """
        idxs = np.random.choice(x.shape[0], size=int(self.max_samples * x.shape[0]), replace=self.bootstrap)
        x_sampled = np.take(x, idxs, axis=0)
        y_sampled = np.take(y, idxs, axis=0)
        return x_sampled, y_sampled

In [3]:
class FeaturesSampler(BaseSampler):
    def __init__(self, axis=1, max_samples=1.0, bootstrap=False):
        self.axis = axis
        super().__init__(max_samples=max_samples, bootstrap=bootstrap)
        
    def sample(self, x):
        """
        Parameters
        ----------
        x : numpy ndarray of shape (n_objects, n_features)
        
        Returns
        -------
        indices : numpy ndarrays of shape (n_features_sampled)
        """
        indices = np.random.choice(x.shape[1], size=int(self.max_samples * x.shape[1]), replace=self.bootstrap)
        return indices

Проверим себя

In [4]:
some_X = np.array([[0, 1, 2], [0.3, 1, 3], [0.5, 1, 3], [1, 2, 1]])
some_y = np.array([1, 5, 3, 1])

object_sampler = ObjectSampler(max_samples=0.7)
feature_sampler = FeaturesSampler(max_samples=0.7)

assert object_sampler.sample(some_X, some_y)[0].shape == (int(0.7*some_X.shape[0]), some_X.shape[1])
assert object_sampler.sample(some_X, some_y)[1].shape == (int(0.7*some_y.shape[0]),)

sample_X, sample_y = object_sampler.sample(some_X, some_y)

for sub_x, sub_y in zip(sample_X, sample_y):
    assert sub_x.tolist() in some_X.tolist()
    assert sub_y in sample_y

assert feature_sampler.sample(some_X).shape == (int(0.7*some_X.shape[1]),)

## 2. Бэггинг

Суть бэггинга заключается в обучении нескольких 'слабых' базовых моделей и объединении их в одну модель, обладающую бОльшей обобщающей способностью. Каждая базовая модель обучается на случайно выбранном подмножестве объектов и на случайно выбранном подмножестве признаков для этих объектов.

Реализуем класс `Bagger`

In [5]:
class Bagger:
    def __init__(
        self, base_estimator,
        object_sampler, feature_sampler,
        n_estimators=10, **params
    ):
        """
        n_estimators : int
            number of base estimators
        base_estimator : class
            class for base_estimator with fit(), predict() and predict_proba() methods
        feature_sampler : instance of FeaturesSampler
        object_sampler : instance of ObjectSampler
        estimators : list
            list for containing base_estimator instances
        indices : list
            list for containing feature indices for each estimator
        params : kwargs
            params for base_estimator initialization
        """
        self.n_estimators = n_estimators
        self.base_estimator = base_estimator
        self.feature_sampler = feature_sampler
        self.object_sampler = object_sampler
        self.estimators = []
        self.indices = []
        self.params = params
    
    def fit(self, X, y):
        """
        for i in range(self.n_estimators):
            1) select random indices of features for current estimator
            2) select random objects and answers for train
            3) fit base_estimator (don't forget to remain only selected features)
            4) save base_estimator (self.estimators) and feature indices (self.indices)
        
        NOTE that self.base_estimator is class and you should init it with
        self.base_estimator(**self.params) before fitting
        """
        for i in range(self.n_estimators):
            indices = self.feature_sampler.sample(X)
            X_sample, y_sample = self.object_sampler.sample(X, y)
            estimator = self.base_estimator(**self.params)
            estimator.fit(np.take(X, indices, axis=1), y)
            self.estimators.append(estimator)
            self.indices.append(indices)
    
    def predict_proba(self, X):
        """
        Returns
        -------
        probas : numpy ndarrays of shape (n_objects, n_classes)
        
        Calculate mean value of all probas from base_estimators
        Don't forget, that each estimator has its own feature indices for prediction
        """
        probas = np.array([estim.predict_proba(np.take(X, idxs, axis=1)) for estim, idxs in 
                          zip(self.estimators, self.indices)])
        probas = np.mean(probas, axis=0)
        return probas
    
    def predict(self, X):
        """
        Returns
        -------
        predictions : numpy ndarrays of shape (n_objects, )
        
        """
        probas = self.predict_proba(X)
        predictions = np.argmax(probas, axis=1)
        return predictions

Для проверки, обучим бэггинг над решающими деревьями (случайный лес)

In [6]:
class RandomForestClassifier(Bagger):
    def __init__(self, n_estimators=30, max_depth=None, min_samples_leaf=1):
        base_estimator = DecisionTreeClassifier
        objects_sampler = ObjectSampler(max_samples=0.9)
        features_sampler = FeaturesSampler(max_samples=0.8)
        
        super().__init__(
            base_estimator=base_estimator,
            object_sampler=object_sampler,
            feature_sampler=feature_sampler,
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_leaf=min_samples_leaf
        )

In [7]:
some_random_forest = RandomForestClassifier()

some_X, some_y = make_classification(n_samples=30, n_features=50,
                                     n_informative=50, n_redundant=0,
                                     random_state=0, shuffle=False)

some_random_forest.fit(some_X, some_y)
predictions = some_random_forest.predict(some_X)
assert isinstance(predictions, type(np.zeros(0)))
npt.assert_equal(predictions, some_y)

## 3. Градиентный бустинг

Бустинг последовательно обучает набор базовых моделей таким образом, что каждая следующая модель пытается исправить ошибки работы предыдущей модели. Логика того, как учитываются ошибки предыдущей модели может быть разной. В алгоритме градиентного бустинга каждая следующая модель обучается на "невязках" предыдущей модели, минимизируя итоговую функцию потерь. У каждого следующего алгоритма вычисляется вес $\alpha$, с которым он входит в ансамбль. Также есть параметр скорости обучения (learning rate), который не позволяет алгоритму переобучиться. Вес $\alpha$ можно находить, используя одномерную оптимизацию. Можно записать процедуру обучения по шагам (будем рассматривать случай бинарной классификации c метками классов {0,1}, чтобы не усложнять жизнь):
1. Настройка базового алгоритма $b_0$.
    
    Алгоритм настраиваются на $y$ с помощью функции MSE.
    
    
2. Будем обозначать текущий небазовый алгоритм - $a$:
    
    $$a_i(x) = \sum_{j=0}^i \alpha_j b_j(x) $$
    
3. Настройка базового алгоритма $b_i$ (обычно это регрессионное дерево):
    
    $$b_i = \arg \min_b \sum_{j=1}^l (b(x_j) + \nabla L(a_{i-1}(x_j), y))^2,$$
    т.е. выход очередного базового алгоритма подстраивается под антиградиент функции потерь
    
4. Настройка веса базового алгоритма $\alpha_i$:
    
    $$\alpha_i = \min_{\alpha > 0} \sum_{j=1}^l L(a_{i-1} + \alpha b_i(x_j), y) $$
    
В случае классфикации будем использовать логистическую функцию потерь. Немного упростим ее:

$$L = -y\log\sigma(a) - (1-y)\log(1 - \sigma(a)) = -\log(1 - \sigma(a)) - y \log \frac{\sigma(a)}{1 - \sigma(a)},$$
где $\sigma$ - функция сигмоиды. Ответ после очередного базового алгоритма надо прогонять через сигмоиду, т.к. не гарантируется, что ответы будут лежать на [0,1] - в этом особенность базового алгоритма (который является регрессионным).

Преобразуем:
$$\log (1 - \sigma(a)) = \log \frac{1}{1 + \exp(a)} = -\log(1 + \exp(a)) $$

$$\log (\frac{\sigma(a)}{1 - \sigma(a)}) = \log(\exp(a)) = a $$
 
Таким образом:

$$L = -ya + \log(1 + \exp(a))$$

Тогда будем вычислять градиент как:
 
$$\nabla L = - y + \sigma(a)$$

Реализуем стратегию обучения базовых классификаторов для класса `GradientBoostingClassifier`

А также класс `BoosterClassifier`

In [8]:
class BoosterClassifier:
    def __init__(
        self, base_estimator, feature_sampler,
        n_estimators=10, lr=.5, **params
    ):
        """
        n_estimators : int
            number of base estimators
        base_estimator : class
            class for base_estimator with fit(), predict() and predict_proba() methods
        feature_sampler : instance of FeaturesSampler
        estimators : list
            list for containing base_estimator instances
        indices : list
            list for containing feature indices for each estimator
        weights : list
            list for containing estimators weights
        lr : float
            learning rate for estimators
        params : kwargs
            kwargs for base_estimator init
        """
        self.n_estimators = n_estimators
        self.base_estimator = base_estimator
        self.feature_sampler = feature_sampler
        self.estimators = []
        self.indices = []
        self.lr = lr
        self.params = params
        self.weights = []
    
    def _fit_first_estimator(self, X, y):
        raise NotImplementedError
        
    def _fit_base_estimator(self, X, y):
        raise NotImplementedError
    
    def fit(self, X, y):
        """
        X : X : numpy ndarray of shape (n_objects, n_features)
        y : numpy ndarray of shape (n_objects, )
        -------
        iteratively fits base models
        """
        self._fit_first_estimator(X, y)
        predictions_base = self.estimators[-1].predict(np.take(X, self.indices[-1], axis=1)) * self.weights[0]
        for i in range(self.n_estimators - 1):
            predictions_base = self._fit_base_estimator(X, y, predictions_base)
    
    def predict_proba(self, X):
        """
        Returns
        -------
        probas : numpy ndarray of shape (n_objects, )
            predictions for one class
        -------
        1) get predictions by first model (self.estimators[0])
        2) for each estimator in self.estimators[1:] (and its indicies and weights):
            update predictions
        3) turn into probability distribution with sigmoid function
        """
        probas = 0
        for estimator, weight, idxs in zip(self.estimators, self.weights, self.indices):
            probas += weight * estimator.predict(np.take(X, idxs, axis=1))
        probas = expit(probas)
        return probas
    
    def predict(self, X):
        probas = self.predict_proba(X)
        return (probas > 0.5).astype(int)

In [9]:
class GradientBoostingClassifier(BoosterClassifier):
    
    def __init__(self, n_estimators=30, lr=0.5, max_depth=2, min_samples_leaf=1):
        """
        n_estimators : int
            number of base estimators
        base_estimator : class
            class for base_estimator with fit(), predict() and predict_proba() methods
        feature_sampler : instance of FeaturesSampler
        estimators : list
            list for containing base_estimator instances
        indices : list
            list for containing feature indices for each estimator
        lr : float
            learning rate for estimators
        params : kwargs
            kwargs for base_estimator init
        """
        
        base_estimator = DecisionTreeRegressor
        feature_sampler = FeaturesSampler(max_samples=0.8)
        super().__init__(
            base_estimator=base_estimator,
            feature_sampler=feature_sampler,
            n_estimators=n_estimators,
            lr=lr,
            max_depth=max_depth,
            min_samples_leaf=min_samples_leaf
        )
        
    def log_loss_gradient(self, y, pred):
        """
        Returns
        -------
        gradient : numpy ndarrays of shape (n_objects, )
        """
        gradient = expit(pred) - y
        return gradient
    
    def log_loss(self, y, pred):
        """
        Returns
        -------
        loss : log loss for inputs
        """
        loss = -y * pred + np.logaddexp(0, pred)
        return loss
    
    def _fit_first_estimator(self, X, y):
        """
        1) select random indices of features for first estimator
        2) fit base_estimator (don't forget to remain only selected features)
        3) calculate optimal weight for estimator by optimization
        4) save base_estimator (self.estimators), feature indices (self.indices) and weight (self.weights)
        
        NOTE that self.base_estimator is class and you should init it with
        self.base_estimator(**self.params) before fitting
        """
        idxs = self.feature_sampler.sample(X)
        estimator = self.base_estimator(**self.params).fit(np.take(X, idxs, axis=1), y)
        weight = 1
        
        self.weights.append(weight)
        self.indices.append(idxs)
        self.estimators.append(estimator)
    
    def _fit_base_estimator(self, X, y, predictions_base):
        """
        X : numpy ndarrays of shape (n_objects, n_features)
        y : numpy ndarrays of shape (n_objects, )
        predictions_base : numpy ndarrays of shape (n_objects, n_classes)
            updated predictions from previous step
        -------
        Returns
        -------
        y_updated : numpy ndarrays of shape (n_objects, n_classes)
            updated predictions
        -------
        
        1) calculate gradient
        2) select random indices of features for current estimator
        3) fit estimator with predictions_base as target
        4) calculate optimal weight for estimator by optimization
        5) calculate y_updated
        6) save estimator, indicies and weights
        
        NOTE that self.base_estimator is class and you should init it with
        self.base_estimator(**self.params) before fitting
        """
        gradient = self.log_loss_gradient(y, predictions_base)
        idxs = self.feature_sampler.sample(X)
        estimator = self.base_estimator(**self.params).fit(np.take(X, idxs, axis=1), gradient)
        pred = estimator.predict(np.take(X, idxs, axis=1))
        func_to_optimize = lambda x: np.sum(self.log_loss(y, predictions_base + x * pred))
        weight = minimize(func_to_optimize, 1).x[0] * self.lr
        y_updated = predictions_base + weight * pred
        self.estimators.append(estimator)
        self.weights.append(weight)
        self.indices.append(idxs)
        return y_updated

Проверим себя

In [10]:
some_gradient_classifier = GradientBoostingClassifier()
some_gradient_classifier.fit(some_X, some_y)
predictions = some_gradient_classifier.predict(some_X)
assert isinstance(predictions, type(np.zeros(0)))
npt.assert_equal(predictions, some_y)

## Эксперименты

Скачаем датасейт для экспериментов [отсюда](https://www.kaggle.com/jsphyg/weather-dataset-rattle-package)

In [149]:
import pandas as pd

In [150]:
data = pd.read_csv('weatherAUS.csv')

Выделим признаки год/месяц/день:

In [151]:
data.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RISK_MM,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,0.0,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,0.0,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,0.0,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,1.0,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,0.2,No


In [152]:
data['year'] = data['Date'].apply(lambda x: x.split('-')[0])
data['month'] = data['Date'].apply(lambda x: x.split('-')[1])
data['day'] = data['Date'].apply(lambda x: x.split('-')[2])

In [153]:
data[['year', 'month', 'day']].head()

,year,month,day
0,2008,12,01
1,2008,12,02
2,2008,12,03
3,2008,12,04
4,2008,12,05


Посмотрим какие года есть в выборке:

In [154]:
data['year'].value_counts()

2016    17508
2014    17400
2015    17231
2009    16595
2010    16419
2013    16097
2011    15126
2012    15044
2017     8466
2008     2246
2007       61
Name: year, dtype: int64

Разделим выборку на три части (train, val и test) по временному принципу:
    
* train - 2007-2014
* val - 2015
* test - 2016-2017

In [155]:
data['year'] = data['year'].astype(int)
data['month'] = data['month'].astype(int)
data['day'] = data['day'].astype(int)

indexes = {
    'train': (data['year'] >= 2007) & (data['year'] <= 2014),
    'val': data['year'] == 2015,
    'test': (data['year'] == 2016) | (data['year'] == 2017)
}

Преобразуем признаки

In [156]:
data.drop(['Date'], axis=1, inplace=True)

In [157]:
data.head()

,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,...,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RISK_MM,RainTomorrow,year,month,day
0,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,WNW,...,8.0,NaN,16.9,21.8,No,0.0,No,2008,12,1
1,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,WSW,...,NaN,NaN,17.2,24.3,No,0.0,No,2008,12,2
2,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,WSW,...,NaN,2.0,21.0,23.2,No,0.0,No,2008,12,3
3,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,E,...,NaN,NaN,18.1,26.5,No,1.0,No,2008,12,4
4,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,NW,...,7.0,8.0,17.8,29.7,No,0.2,No,2008,12,5


In [158]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142193 entries, 0 to 142192
Data columns (total 26 columns):
Location         142193 non-null object
MinTemp          141556 non-null float64
MaxTemp          141871 non-null float64
Rainfall         140787 non-null float64
Evaporation      81350 non-null float64
Sunshine         74377 non-null float64
WindGustDir      132863 non-null object
WindGustSpeed    132923 non-null float64
WindDir9am       132180 non-null object
WindDir3pm       138415 non-null object
WindSpeed9am     140845 non-null float64
WindSpeed3pm     139563 non-null float64
Humidity9am      140419 non-null float64
Humidity3pm      138583 non-null float64
Pressure9am      128179 non-null float64
Pressure3pm      128212 non-null float64
Cloud9am         88536 non-null float64
Cloud3pm         85099 non-null float64
Temp9am          141289 non-null float64
Temp3pm          139467 non-null float64
RainToday        140787 non-null object
RISK_MM          142193 non-null floa

### дропнем колонки с большим количеством пропусков

In [159]:
data.drop(['Evaporation', 'Sunshine', 'Cloud9am', 'Cloud3pm'], axis=1, inplace=True)

In [160]:
data.isna().any()

Location         False
MinTemp           True
MaxTemp           True
Rainfall          True
WindGustDir       True
WindGustSpeed     True
WindDir9am        True
WindDir3pm        True
WindSpeed9am      True
WindSpeed3pm      True
Humidity9am       True
Humidity3pm       True
Pressure9am       True
Pressure3pm       True
Temp9am           True
Temp3pm           True
RainToday         True
RISK_MM          False
RainTomorrow     False
year             False
month            False
day              False
dtype: bool

In [161]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142193 entries, 0 to 142192
Data columns (total 22 columns):
Location         142193 non-null object
MinTemp          141556 non-null float64
MaxTemp          141871 non-null float64
Rainfall         140787 non-null float64
WindGustDir      132863 non-null object
WindGustSpeed    132923 non-null float64
WindDir9am       132180 non-null object
WindDir3pm       138415 non-null object
WindSpeed9am     140845 non-null float64
WindSpeed3pm     139563 non-null float64
Humidity9am      140419 non-null float64
Humidity3pm      138583 non-null float64
Pressure9am      128179 non-null float64
Pressure3pm      128212 non-null float64
Temp9am          141289 non-null float64
Temp3pm          139467 non-null float64
RainToday        140787 non-null object
RISK_MM          142193 non-null float64
RainTomorrow     142193 non-null object
year             142193 non-null int32
month            142193 non-null int32
day              142193 non-null int32

### разобьем колонки на категории и чиселки
заполним пропуски в категориях значением 'unknown'

а чиселки заполним средним

In [162]:
cat_columns = ['Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm', 'RainToday', 'RainTomorrow']
num_columns = list(set(data.columns) - set(cat_columns))

In [163]:
data[num_columns] = data[num_columns].fillna(data[num_columns].mean())

In [164]:
data[cat_columns] = data[cat_columns].fillna('Unknown')

In [165]:
data.isna().any()

Location         False
MinTemp          False
MaxTemp          False
Rainfall         False
WindGustDir      False
WindGustSpeed    False
WindDir9am       False
WindDir3pm       False
WindSpeed9am     False
WindSpeed3pm     False
Humidity9am      False
Humidity3pm      False
Pressure9am      False
Pressure3pm      False
Temp9am          False
Temp3pm          False
RainToday        False
RISK_MM          False
RainTomorrow     False
year             False
month            False
day              False
dtype: bool

In [166]:
data['Location'].unique()

array(['Albury', 'BadgerysCreek', 'Cobar', 'CoffsHarbour', 'Moree',
       'Newcastle', 'NorahHead', 'NorfolkIsland', 'Penrith', 'Richmond',
       'Sydney', 'SydneyAirport', 'WaggaWagga', 'Williamtown',
       'Wollongong', 'Canberra', 'Tuggeranong', 'MountGinini', 'Ballarat',
       'Bendigo', 'Sale', 'MelbourneAirport', 'Melbourne', 'Mildura',
       'Nhil', 'Portland', 'Watsonia', 'Dartmoor', 'Brisbane', 'Cairns',
       'GoldCoast', 'Townsville', 'Adelaide', 'MountGambier', 'Nuriootpa',
       'Woomera', 'Albany', 'Witchcliffe', 'PearceRAAF', 'PerthAirport',
       'Perth', 'SalmonGums', 'Walpole', 'Hobart', 'Launceston',
       'AliceSprings', 'Darwin', 'Katherine', 'Uluru'], dtype=object)

### приведем object to cat, чтобы можно было использовать лейбл энкодинг из коробки пандаса

In [167]:
data[cat_columns] = data[cat_columns].astype('category', inplace=True)

In [168]:
for c in cat_columns:
    data[c] = data[c].cat.codes

### убеждаемся, что всё закодировалось как надо

In [169]:
data.head()

,Location,MinTemp,MaxTemp,Rainfall,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,...,Pressure9am,Pressure3pm,Temp9am,Temp3pm,RainToday,RISK_MM,RainTomorrow,year,month,day
0,2,13.4,22.9,0.6,14,44.0,14,15,20.0,24.0,...,1007.7,1007.1,16.9,21.8,0,0.0,0,2008,12,1
1,2,7.4,25.1,0.0,15,44.0,6,16,4.0,22.0,...,1010.6,1007.8,17.2,24.3,0,0.0,0,2008,12,2
2,2,12.9,25.7,0.0,16,46.0,14,16,19.0,26.0,...,1007.6,1008.7,21.0,23.2,0,0.0,0,2008,12,3
3,2,9.2,28.0,0.0,4,24.0,9,0,11.0,9.0,...,1017.6,1012.8,18.1,26.5,0,1.0,0,2008,12,4
4,2,17.5,32.3,1.0,14,41.0,1,7,7.0,20.0,...,1010.8,1006.0,17.8,29.7,0,0.2,0,2008,12,5


Таргет - `RainTommorow`. Удалим его из обучающих данных, также удалим признак `RISK_MM`.

In [170]:
target_data = data['RainTomorrow']
data.drop(['RainTomorrow', 'RISK_MM'], axis=1, inplace=True)

разбиваем на траин, вал и тест

In [171]:
X_train, y_train = data[indexes['train']], target_data[indexes['train']]
X_val, y_val = data[indexes['val']], target_data[indexes['val']]
X_test, y_test = data[indexes['test']], target_data[indexes['test']]

первая проба

In [53]:
some_gradient_classifier = GradientBoostingClassifier()
some_gradient_classifier.fit(X_train, y_train)
predictions = some_gradient_classifier.predict(X_val)

Посмотрим на работу алгоритма при разном количестве эстиматоров

In [37]:
from sklearn.metrics import accuracy_score as score

In [52]:
score(y_val, predictions)

0.8447565434391504

In [54]:
predictions = some_gradient_classifier.predict(X_test)
score(predictions, y_test)

0.8369138369138369

In [55]:
some_gradient_classifier = GradientBoostingClassifier(n_estimators=100)
some_gradient_classifier.fit(X_train, y_train)

In [56]:
predictions = some_gradient_classifier.predict(X_val)
score(predictions, y_val)

0.8524171551273867

In [57]:
predictions = some_gradient_classifier.predict(X_test)
score(predictions, y_test)

0.8434588434588435

In [58]:
some_gradient_classifier = GradientBoostingClassifier(n_estimators=100, lr=.25)
some_gradient_classifier.fit(X_train, y_train)

In [59]:
predictions = some_gradient_classifier.predict(X_val)
score(predictions, y_val)

0.8504439672682955

In [60]:
predictions = some_gradient_classifier.predict(X_test)
score(predictions, y_test)

0.8411488411488411

In [61]:
some_gradient_classifier = GradientBoostingClassifier(n_estimators=200)
some_gradient_classifier.fit(X_train, y_train)

In [62]:
predictions = some_gradient_classifier.predict(X_val)
score(predictions, y_val)

0.8559572862863444

In [63]:
predictions = some_gradient_classifier.predict(X_test)
score(predictions, y_test)

0.8457303457303458

### подберем оптимальный learning rate

In [64]:
lrs = [0.1 * i for i in range(1, 11)]

In [67]:
scores = []
for lr in lrs:
    some_gradient_classifier = GradientBoostingClassifier(n_estimators=200, lr=lr)
    some_gradient_classifier.fit(X_train, y_train)
    predictions = some_gradient_classifier.predict(X_val)
    scores.append(score(predictions, y_val))

In [69]:
np.argmax(scores)

3

In [70]:
scores

[0.8502118275201671,
 0.8533457141199002,
 0.8522430503162904,
 0.855144797167895,
 0.8539840984272532,
 0.8542742731124137,
 0.8539260634902212,
 0.8548546224827346,
 0.8539840984272532,
 0.8540421333642852]

In [72]:
lrs[3]

0.4

### оптимальные параметры для градиентного бустинга ниже

In [73]:
some_gradient_classifier = GradientBoostingClassifier(n_estimators=400, lr=0.4)
some_gradient_classifier.fit(X_train, y_train)

In [ ]:
predictions = some_gradient_classifier.predict(X_val)
score(predictions, y_val)

In [75]:
predictions = some_gradient_classifier.predict(X_test)
score(predictions, y_test)

0.8475398475398476

### подберем теперь случайный лес

In [81]:
some_random_forest = RandomForestClassifier()
some_random_forest.fit(X_train, y_train)

In [82]:
predictions = some_random_forest.predict(X_val)
score(predictions, y_val)

0.8473681156055946

In [84]:
predictions = some_random_forest.predict(X_test)
score(predictions, y_test)

0.8375298375298376

In [85]:
some_random_forest = RandomForestClassifier(n_estimators=200)
some_random_forest.fit(X_train, y_train)

In [86]:
predictions = some_random_forest.predict(X_val)
score(predictions, y_val)

0.8521850153792583

In [87]:
predictions = some_random_forest.predict(X_test)
score(predictions, y_test)

0.8421113421113421

### оптимальные параметры алгоритмов и результат их работы

In [172]:
some_gradient_classifier = GradientBoostingClassifier(n_estimators=400, lr=0.4)
some_gradient_classifier.fit(X_train, y_train)
predictions_val = some_gradient_classifier.predict(X_val)
score_val = score(predictions_val, y_val)
predictions_test = some_gradient_classifier.predict(X_test)
score_test = score(predictions_test, y_test)
print(f'Gradient Boosting Classifier:\nValidation accuracy = {score_val}\nTest accuracy = {score_test}')

Gradient Boosting Classifier:
Validation accuracy = 0.8559572862863444
Test accuracy = 0.8463078463078463


In [173]:
assert(score_val > 0.845 and score_test > 0.845)

In [174]:
some_random_forest = RandomForestClassifier(n_estimators=200)
some_random_forest.fit(X_train, y_train)
predictions_val = some_random_forest.predict(X_val)
score_val = score(predictions_val, y_val)
predictions_test = some_random_forest.predict(X_test)
score_test = score(predictions_test, y_test)
print(f'Random Forest Classifier:\nValidation accuracy = {score_val}\nTest accuracy = {score_test}')

Random Forest Classifier:
Validation accuracy = 0.8534037490569323
Test accuracy = 0.8423808423808424


In [175]:
assert(score_val > 0.84 and score_test > 0.84)

## 3.1 AdaBoost

В алгоритме AdaBoost всем объектам обучения присваивается вес `weight`, который определяет степень важности объекта при обучении. И если текущая модель ошибается на некотором объекте, его вес увеличивается, и этот объект будет больше влиять на обучение следующей модели. Также, так как модели обучаются последовательно, они не равносильны между собой, поэтому у каждой модели тоже есть вес `alpha`, который определяет вес модели при суммировании ответов и зависит от количества ошибок `err` модели. На каждой итерации обучения, эти веса пересчитываются по формулам:

* $$\alpha_j = \log\left(\frac{1-err_j}{err_j}\right),$$
где $err_j$ - ошибка классификации

* $$w_{new}^t = \frac{w_{old}^{t}*\exp(-\alpha_j \cdot y(x^t) \cdot b_j(x^t))}{\sum\limits_{i=1}^m w_{old}^{t}*\exp(-\alpha_j \cdot y(x^i) \cdot b_j(x^i))}$$
Изначально все веса объектов $w^i$ равны (и нормированы на 1).

Реализуем AdaBoost, учтя следующее:
* надо работать с метками {-1,1} - это обусловлено использованием экспоненциальной функции потерь
* метод `predict` представляет собой функцию сигмоид, примененную к сумме предсказаний всех моделей  

In [142]:
class AdaboostClassifier:
    estimators = []
    weights = []
    def __init__(self, n_estimators=10, learning_rate=0.5):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
    
    
    def fit(self, X, y):
        # initial weights for samples
        w = np.ones(X.shape[0]) / X.shape[0]
        
        # fit each estimator
        for i in range(self.n_estimators):
            # fit base estimator
            clf = DecisionTreeClassifier(max_depth=1).fit(X, y, sample_weight=w)
            
            # get predict from estimator
            preds = clf.predict(X)
            
            # indicator
            miss = (preds != y).astype(int)
            
            # indicator -1, 1 to do exponent right
            miss2 = np.where(miss == 0, -1, 1)
            
            # error
            error = w @ miss / np.sum(w)
            
            # calculate weight of estimator
            weight = self.learning_rate * np.log((1 - error) / error)
            
            # update weights
            w = w * np.exp(miss2 * weight)
            
            # append estimator and weight to classifier
            self.estimators.append(clf)
            self.weights.append(weight)
    
    
    def predict(self, X):
        preds = np.zeros(X.shape[0])
        for estimator, weight in zip(self.estimators, self.weights):
            preds += weight * estimator.predict(X)
        return np.sign(preds)

###### Основные идеи имплементации взяты из [этой](https://towardsdatascience.com/machine-learning-part-17-boosting-algorithms-adaboost-in-python-d00faac6c464) статьи.

Вспомним, что для адабуста нужны метки -1 и 1, а не 0 и 1, поэтому в таргете заменим 0 на -1

In [176]:
y_test[y_test == 0] = -1
y_val[y_val == 0] = -1
y_train[y_train == 0] = -1

In [143]:
adaboost = AdaboostClassifier(n_estimators=2)
adaboost.fit(X_train, y_train)

preds = adaboost.predict(X_train)

print(score(preds, y_train)) #0.815

preds = adaboost.predict(X_val)
print(score(preds, y_val))

preds = adaboost.predict(X_test)
print(score(preds, y_test))
#815, 822, 808

0.815684729462157
0.8220648830596019
0.8084623084623085


In [127]:
adaboost = AdaboostClassifier(n_estimators=100)
adaboost.fit(X_train, y_train)

preds = adaboost.predict(X_train)

score(preds, y_train) #0.815

preds = adaboost.predict(X_val)
score(preds, y_val)

preds = adaboost.predict(X_test)
score(preds, y_test)

0.8351517355639068

In [131]:
adaboost = AdaboostClassifier(n_estimators=200)
adaboost.fit(X_train, y_train)

preds = adaboost.predict(X_train)

print(score(preds, y_train)) #0.815

preds = adaboost.predict(X_val)
print(score(preds, y_val))

preds = adaboost.predict(X_test)
print(score(preds, y_test))

0.8389905847173394
0.8398816087284545
0.8298298298298298


In [146]:
adaboost = AdaboostClassifier(n_estimators=300)
adaboost.fit(X_train, y_train)
print('AdaboostClassifier')
preds = adaboost.predict(X_train)
print(f'Train accuracy = {score(preds, y_train)}')
preds = adaboost.predict(X_val)
print(f'Validation accuracy = {score(preds, y_val)}')
preds = adaboost.predict(X_test)
print(f'Test accuracy = {score(preds, y_test)}')

AdaboostClassifier
Train accuracy = 0.8398189679557118
Validation accuracy = 0.8401137484765829
Test accuracy = 0.8321398321398321


~о май гад, на валидации скор лучше, чем на трейне, кто-нибудь остановите адабуст~

### Оптимальные параметры для adaboost и результат работы алгоритма

In [178]:
adaboost = AdaboostClassifier(n_estimators=300)
adaboost.fit(X_train, y_train)
preds_val = adaboost.predict(X_val)
preds_test = adaboost.predict(X_test)
score_val = score(preds_val, y_val)
score_test = score(preds_test, y_test)
print(f'Adaboost Classifier:\nValidation accuracy = {score_val}\nTest accuracy = {score_test}')

Adaboost Classifier:
Validation accuracy = 0.8423771110208346
Test accuracy = 0.8352198352198352


In [179]:
assert(score_val > 0.83 and score_test > 0.83)

Некоторые наблюдения по адабусту в ходе экспериментов. Дико переобучается, если в выборке есть шумы/выбросы, и это не фиксится даже лернинг рейтом. Но решение есть: обучить, посмотреть распределение весов, если веса большие у объектов, то это скорее всего выбросы, удалить их из выборки, обучить заново на новой выборки (решение, конечно, так себе, т.к. обучаться долго будет...)